In [1]:
# pip unistall ipywidgets
# pip install ipywidgets
# pip install tabulate
# pip install plotly 

In [2]:
import ipywidgets as w
from IPython.display import display, clear_output
import json

from widgets_utils import *
from utils import *

with open('data.txt', 'r') as f:
    parts = json.load(f)

### 1. Requirements

In [98]:
# Requirements | functions

def find_Pareto(parts, restriction):
    
    def combinations(parts, k):
        if k > 1:
            comb = [(*x, y) for x in combinations(parts, k-1) for y in parts[k]]
            return comb
        else:
            comb = [(x, y) for x in parts[k-1] for y in parts[k]]
            return comb
        
    def get_attribute_sum(comb, attr):
        attr_sum = 0
        for part in comb:
            attr_sum += list(part.values())[0][attr]
        return attr_sum
        
    new_parts = []
    for part in get_keys(parts): #restriction.keys():
        particular_parts = filter_parts(parts, part)
        for example in particular_parts:
            example_satisfies_conditions = True
            for attr in restriction[part].keys():
                if isinstance(restriction[part][attr], str):
                    condition = example[part][attr] == restriction[part][attr]
                elif isinstance(restriction[part][attr], tuple):
                    min_value, max_value = restriction[part][attr]
                    condition = example[part][attr] >= min_value and example[part][attr] <= max_value
                elif isinstance(restriction[part][attr], bool):
                    condition = not restriction[part][attr] or (example[part][attr] 
                                                                and restriction[part][attr])
                if not condition:
                    example_satisfies_conditions = False
                    break
            if example_satisfies_conditions:
                new_parts.append(example)

#     print(list(map(lambda part: '{}: {}'.format(part, filter_parts(new_parts, part) != []), get_keys(parts))))
    if not all(filter_parts(new_parts, part) != [] for part in get_keys(parts)):
        return []
        
    new_pparts = []
    for part in get_keys(new_parts):
        new_pparts.append(filter_parts(new_parts, part))
    filtered_combinations = []
    for comb in combinations(new_pparts, len(new_pparts)-1):
        min_cost, max_cost = restriction['Total']['cost']
        min_weight, max_weight = restriction['Total']['weight']
        comb_cost = get_attribute_sum(comb, 'price')
        comb_weight = get_attribute_sum(comb, 'weight')
        if comb_cost >= min_cost and comb_cost <= max_cost and \
        comb_weight >= min_weight and comb_weight <= max_weight:
            filtered_combinations.append(comb)
    return filtered_combinations

def analyse(b, parts):
    restriction = {}
    for key in get_keys(parts):
        restriction[key] = {}
        
    restriction['Body']['material'] = w1.children[1].value
    restriction['Body']['portability'] = w1.children[2].value

    restriction['Engine']['engine_type'] = w2.children[1].value
    restriction['Engine']['number'] = w2.children[2].value
    restriction['Engine']['propulsion_type'] = w2.children[3].value
    restriction['Propeller']['material'] = w2.children[4].value
    restriction['Propeller']['number'] = w2.children[5].value

    restriction['Software']['GPS'] = w3.children[1].value
    restriction['Software']['AI'] = w3.children[2].value
    restriction['Software']['manufacturer'] = w3.children[3].value
    restriction['CPU']['frequency'] = w3.children[4].value
    restriction['CPU']['num_cores'] = w3.children[5].value
    restriction['Sensor']['signal_acceptance_distance'] = w3.children[6].value
    restriction['Camera']['resolution'] = w3.children[7].value
    restriction['Camera']['angle'] = w3.children[8].value
    restriction['Camera']['IR_illumination'] = w3.children[9].value

    restriction['Accumulator']['duration'] = w4.children[1].value
    restriction['Accumulator']['take_off_weight'] = w4.children[2].value
    restriction['Accumulator']['height'] = w4.children[3].value
    
    restriction['Total'] = {}
    restriction['Total']['cost'] = w5.children[1].value
    restriction['Total']['weight'] = w5.children[2].value
    
    Pareto_structures = find_Pareto(parts, restriction)
    visualize_structure(b, Pareto_structures, 1)


def visualize_structure(b, Pareto_structures, i):
    all_widgets = get_template_structures(simplify_data_structure(Pareto_structures))
    window_paretto = w.VBox([widget_header('Lab 5', bold=False, face=facefont),
        widget_header('Дрон. Множество структур Паретто', bold=False, size=5, face=facefont),
        widget_header('(мощность множества: {})'.format(len(Pareto_structures)), bold=False, size=5, face=facefont),
        all_widgets], layout=allLayout)
    
    tab = create_tab(window_requirements, window_paretto, window_priority)
    clear_output()
    display(tab)

In [99]:
# Requirements | Visualization
    
####################################################################################################
# Іміджева складова
####################################################################################################
w1_widgets = [
    widget_typeBox('Материал корпуса', get_part_attr(parts, 'Body', 'material')),
    widget_flag('Портативность')
]
w1 = widget_group('Внешний вид', w1_widgets)

####################################################################################################
# Механічна складова
####################################################################################################
w2_widgets = [
    widget_typeBox('Тип двигателя', get_part_attr(parts, 'Engine', 'engine_type')),
    widget_rangeBoxInt('Число двигателей', *get_boundaries(parts, 'Engine', 'number')),
    widget_typeBox('Тип движителя', get_part_attr(parts, 'Engine', 'propulsion_type')),

    widget_typeBox('Материал пропеллера', get_part_attr(parts, 'Propeller', 'material')),
    widget_rangeBoxInt('Число пропеллеров', *get_boundaries(parts, 'Propeller', 'number')),
]
w2 = widget_group('Механика', w2_widgets)

####################################################################################################
# Апаратна складова
####################################################################################################
w3_widgets = [
    widget_flag('Наличие GPS'),
    widget_flag('Наличие ИИ'),
    widget_typeBox('Производитель ', get_part_attr(parts, 'Software', 'manufacturer')),

    widget_rangeBoxFloat('Частота процессора, Гц', *get_boundaries(parts, 'CPU', 'frequency')),
    widget_rangeBoxInt('Количество ядер процессора', *get_boundaries(parts, 'CPU', 'num_cores')),

    widget_rangeBoxFloat('Дальность приема сигнала, м', *get_boundaries(parts, 'Sensor',
                                                                        'signal_acceptance_distance')),

    widget_rangeBoxFloat('Разрешение матрицы, Мп', *get_boundaries(parts, 'Camera', 'resolution')),
    widget_rangeBoxInt('Угол обзора', *get_boundaries(parts, 'Camera', 'angle')),
    widget_rangeBoxInt('Дальность ИК подсветки, м', *get_boundaries(parts, 'Camera', 'IR_illumination')),
]
w3 = widget_group('Аппаратная составляющая', w3_widgets)

####################################################################################################
# Акумулятор
####################################################################################################
w4_widgets = [
    widget_rangeBoxInt('Продолжительность полета, мин', *get_boundaries(parts, 'Accumulator', 'duration')),
    widget_rangeBoxInt('Взлетная масса, кг', *get_boundaries(parts, 'Accumulator', 'take_off_weight')),
    widget_rangeBoxInt('Высота полета, м', *get_boundaries(parts, 'Accumulator', 'height')),
]
w4 = widget_group('Аккумулятор', w4_widgets)

####################################################################################################
# Total
####################################################################################################
w5_widgets = [
#     widget_rangeBox('Общая стоимость', 0, 1000),
#     widget_rangeBox('Общий вес', 0, 1000)
     widget_rangeBoxInt('Стоимость, грн', 0, 30000),
     widget_rangeBoxInt('Общий вес, г', 0, 1000),
]
layout = w.Layout(width='480px', border='solid 0px', margin='10px',
                  justify_content='center', align_items='center')
w5 = widget_group('', w5_widgets, layout=layout)

b1 = w.Button(
    description='Анализ',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

b1.on_click(lambda b: analyse(b, parts))
all_widgets = w.HBox([w.VBox([w1, w2, w5, b1], layout=bigGroup), w.VBox([w3, w4], layout=bigGroup)])
window_requirements = w.VBox([widget_header('Lab 5', bold=False, face=facefont),
            widget_header('Дрон. Требования к функциональным элементам', bold=False, size=5, face=facefont),
            all_widgets], layout=allLayout) 

### 2. Paretto set of structures

In [105]:
def get_template_structures(combinations):    
    if len(combinations) == 0:
        all_widgets = w.Label('К сожалению, нет структуры, которая бы удовлетворяла текущие требования.',
                       layout=allLayout_without_border)
        return all_widgets
    
    def calculate(rw1, rw2, rw3, rw4, rw5):
        rw5.children[1].children[1].value = str(get_price(parts, rw1.children[1].value,
                                                          rw2.children[1].value, rw2.children[2].value,
                                                          rw3.children[1].value, rw3.children[2].value,
                                                          rw3.children[3].value, rw3.children[4].value,
                                                          rw4.children[1].value)) + ' грн'
        rw5.children[2].children[1].value = str(get_weight(parts, rw1.children[1].value,
                                                           rw2.children[1].value, rw2.children[2].value,
                                                           rw3.children[1].value, rw3.children[2].value,
                                                           rw3.children[3].value, rw3.children[4].value,
                                                           rw4.children[1].value)) + ' г'
    
    ####################################################################################################
    # Іміджева складова
    ####################################################################################################
    rw1_widgets = [
        widget_typeBox('Корпус', get_comb_attr(combinations, 'Body', 'name')),
    ]

    rw1 = widget_group('Внешний вид', rw1_widgets)

    ####################################################################################################
    # Механічна складова
    ####################################################################################################
    rw2_widgets = [
        widget_typeBox('Двигатели', get_comb_attr(combinations, 'Engine', 'name')),

        widget_typeBox('Пропеллеры', get_comb_attr(combinations, 'Propeller', 'name')),
    ]
    rw2 = widget_group('Механика', rw2_widgets)

    ####################################################################################################
    # Апаратна складова
    ####################################################################################################
    rw3_widgets = [
        widget_typeBox('Процессор', get_comb_attr(combinations, 'CPU', 'name')),

        widget_typeBox('Датчик связи', get_comb_attr(combinations, 'Sensor', 'name')),

        widget_typeBox('Программное обеспечение', get_comb_attr(combinations, 'Software', 'name')),
        widget_typeBox('Камера', get_comb_attr(combinations, 'Camera', 'name')),
    ]
    rw3 = widget_group('Аппаратная составляющая', rw3_widgets)

    ####################################################################################################
    # Акумулятор
    ####################################################################################################
    rw4_widgets = [
        widget_typeBox('Аккумулятор', get_comb_attr(combinations, 'Accumulator', 'name')),
    ]
    rw4 = widget_group('Аккумулятор', rw4_widgets)

    ####################################################################################################
    # Total
    ####################################################################################################
    totalLayout = w.Layout(width='200px', justify_content='flex-end')
    rw5_widgets = [
          w.HBox([w.Label(value='Общая стоимость:', style=title_style),
                  w.Label(value='', style=title_style)], layout=totalLayout),
          w.HBox([w.Label(value='Общий вес:', style=title_style),
                  w.Label(value='', style=title_style)], layout=totalLayout),
    ]
    rw5 = widget_group('', rw5_widgets)

    rs1 = w.IntSlider(value=1, min=1, max=len(combinations), step=1,
                      description='Структура')
    
    def set_structure(i, rw1, rw2, rw3, rw4, rw5):
        i -= 1
        rw1.children[1].value = get_comb_attr(combinations, 'Body', 'name')[i]
        rw2.children[1].value = get_comb_attr(combinations, 'Engine', 'name')[i]
        rw2.children[2].value = get_comb_attr(combinations, 'Propeller', 'name')[i]
        rw3.children[1].value = get_comb_attr(combinations, 'CPU', 'name')[i]
        rw3.children[2].value = get_comb_attr(combinations, 'Sensor', 'name')[i]
        rw3.children[3].value = get_comb_attr(combinations, 'Software', 'name')[i]
        rw3.children[4].value = get_comb_attr(combinations, 'Camera', 'name')[i]
        rw4.children[1].value = get_comb_attr(combinations, 'Accumulator', 'name')[i]
        calculate(rw1, rw2, rw3, rw4, rw5)
    
    w.interact(lambda x: set_structure(x, rw1, rw2, rw3, rw4, rw5), x=rs1)
    
    all_widgets = w.HBox([w.VBox([rw1, rw2, rw5], layout=bigGroup),
                          w.VBox([rw3, rw4, rs1], layout=bigGroup)],
                         layout=allLayout_without_border)
    return all_widgets

In [106]:
all_widgets = w.Label('')
window_paretto = w.VBox([widget_header('Lab 5', bold=False, face=facefont),
        widget_header('Дрон. Множество структур Паретто', bold=False, size=5, face=facefont),
        all_widgets], layout=allLayout) 

### 3. Prior requirement

In [107]:
all_widgets = w.Label('')
window_priority = w.VBox([widget_header('Lab 5', bold=False, face=facefont),
        widget_header('Дрон. Приоритетное требование', bold=False, size=5, face=facefont),
        all_widgets], layout=allLayout) 

In [108]:
def create_tab(window_requirements, window_paretto, window_priority):
    tab_titles = ['Требования', 'Множество структур Паретто', 'Приоритетное требование']
    children = [window_requirements, window_paretto, window_priority]
    tab = w.Tab()
    tab.children = children
    for i in range(len(children)):
        tab.set_title(i, tab_titles[i])
    return tab

In [109]:
tab = create_tab(window_requirements, window_paretto, window_priority)
display(tab)

Tab(children=(VBox(children=(HTML(value="<div align='center'><font size='6px' face='Arial' color='#0969A2'>Lab 5</font></div>"), HTML(value="<div align='center'><font size='5px' face='Arial' color='#0969A2'>Дрон. Требования к функциональным элементам</font></div>"), HBox(children=(VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Внешний вид</font></div>"), Dropdown(description='Материал корпуса', index=3, layout=Layout(justify_content='center', width='400px'), options=('Волоконный композит', 'Кевлар', 'Углепластика', 'Экструдированный пенополистирол'), style=DescriptionStyle(description_width='initial'), value='Экструдированный пенополистирол'), Checkbox(value=False, description='Портативность', style=DescriptionStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Механика</font></div>"), Dropdown(description='Тип двигателя', layout=Layout(justify_content='center', width='400px'), options=('Бесколлекторный электродвигатель', 'Коллекторный электродвигатель', 'Электродвигатель'), style=DescriptionStyle(description_width='initial'), value='Бесколлекторный электродвигатель'), FloatRangeSlider(value=(2.0, 5.0), continuous_update=False, description='Число двигателей', layout=Layout(justify_content='flex-start', width='400px'), max=6.0, min=2.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial')), Dropdown(description='Тип движителя', layout=Layout(justify_content='center', width='400px'), options=('Винт', 'Реактивная тяга'), style=DescriptionStyle(description_width='initial'), value='Винт'), Dropdown(description='Материал пропеллера', layout=Layout(justify_content='center', width='400px'), options=('Волоконный композит', 'Карбон'), style=DescriptionStyle(description_width='initial'), value='Волоконный композит'), FloatRangeSlider(value=(2.0, 5.0), continuous_update=False, description='Число пропеллеров', layout=Layout(justify_content='flex-start', width='400px'), max=6.0, min=2.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'></font></div>"), FloatRangeSlider(value=(0.0, 30000.0), continuous_update=False, description='Стоимость, грн', layout=Layout(justify_content='flex-start', width='400px'), max=30000.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial')), FloatRangeSlider(value=(0.0, 1000.0), continuous_update=False, description='Общий вес, г', layout=Layout(justify_content='flex-start', width='400px'), max=1000.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='10px', width='480px')), Button(description='Анализ', style=ButtonStyle())), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px')), VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Аппаратная составляющая</font></div>"), Checkbox(value=False, description='Наличие GPS', style=DescriptionStyle(description_width='initial')), Checkbox(value=False, description='Наличие ИИ', style=DescriptionStyle(description_width='initial')), Dropdown(description='Производитель ', layout=Layout(justify_content='center', width='400px'), options=('Agisoft', 'Harris Geospatial Solutions', 'Trimble'), style=DescriptionStyle(description_width='initial'), value='Agisoft'), FloatRangeSlider(value=(2.1, 3.0), continuous_update=False, description='Частота процессора, Гц', layout=Layout(justify_content='flex-start

### Ответ

In [10]:
combinations = [({'Accumulator': {'name': 'Luftera', 'price': 1400, 'weight': 200, 'duration': 360,
                                  'take_off_weight': 5, 'height': 150}},
                 {'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True,
                               'AI': False, 'manufacturer': 'Agisoft'}},
                 {'Propeller': {'name': 'Phantom 3', 'price': 200, 'weight': 30,
                                'material': 'Волоконный композит', 'number': 2}},
                 {'Camera': {'name': 'AVT 6600AUXSDWF', 'price': 1750, 'weight': 200, 'resolution': 2.4, 
                             'angle': 180, 'IR_illumination': 35}}, 
                 {'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 
                             'signal_acceptance_distance': 1}}, 
                 {'Engine': {'name': 'Syma NHYDXS', 'price': 1600, 'weight': 330, 
                             'engine_type': 'Бесколлекторный электродвигатель', 'number': 2, 
                             'propulsion_type': 'Винт'}}, 
                 {'CPU': {'name': 'TMS320C6678', 'price': 460, 'weight': 0, 
                          'frequency': 1.8, 'num_cores': 6}}, 
                 {'Body': {'name': 'AgDrone', 'price': 15000, 'weight': 180, 
                           'portability': True, 'material': 'Волоконный композит'}})]

def simplify_data_structure(combinations):
    new_combinations = []
    for combination in combinations:
        new_combination = {}
        for element in combination:
            new_combination[list(element.keys())[0]] = list(element.values())[0]
        new_combinations.append(new_combination)
    return new_combinations

combinations = simplify_data_structure(combinations)
display(get_template_structure(combinations))

HBox(children=(VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Зовнішній вигляд</font></div>"), Dropdown(description='Корпус', layout=Layout(justify_content='center', width='400px'), options=('AgDrone',), style=DescriptionStyle(description_width='initial'), value='AgDrone')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Механіка</font></div>"), Dropdown(description='Двигуни', layout=Layout(justify_content='center', width='400px'), options=('Syma NHYDXS',), style=DescriptionStyle(description_width='initial'), value='Syma NHYDXS'), Dropdown(description='Проперели', layout=Layout(justify_content='center', width='400px'), options=('Phantom 3',), style=DescriptionStyle(description_width='initial'), value='Phantom 3')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'></font></div>"), HBox(children=(Label(value='Загальна вартість:', style=DescriptionStyle(description_width='initial')), Label(value='20770 грн', style=DescriptionStyle(description_width='initial'))), layout=Layout(justify_content='flex-end', width='200px')), HBox(children=(Label(value='Загальна вага:', style=DescriptionStyle(description_width='initial')), Label(value='940 г', style=DescriptionStyle(description_width='initial'))), layout=Layout(justify_content='flex-end', width='200px'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px'))), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px')), VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Апаратна складова</font></div>"), Dropdown(description='Процесор', layout=Layout(justify_content='center', width='400px'), options=('TMS320C6678',), style=DescriptionStyle(description_width='initial'), value='TMS320C6678'), Dropdown(description="Датчик зв'язку", layout=Layout(justify_content='center', width='400px'), options=('Namur',), style=DescriptionStyle(description_width='initial'), value='Namur'), Dropdown(description='Програмне забезпечення', layout=Layout(justify_content='center', width='400px'), options=('Agisoft Photoscan',), style=DescriptionStyle(description_width='initial'), value='Agisoft Photoscan'), Dropdown(description='Камера', layout=Layout(justify_content='center', width='400px'), options=('AVT 6600AUXSDWF',), style=DescriptionStyle(description_width='initial'), value='AVT 6600AUXSDWF')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Акумулятор</font></div>"), Dropdown(description='Акумулятор', layout=Layout(justify_content='center', width='400px'), options=('Luftera',), style=DescriptionStyle(description_width='initial'), value='Luftera')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), Button(description='Прорахувати', style=ButtonStyle())), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px'))), layout=Layout(align_items='stretch', border='solid 0px', justify_content='center', padding='20px'))

In [ ]:
parts = [['aa', 'baa', 'caa'], ['dgg', 'fgg'], ['1', '2ss'], ['3', '4']]
combinations(parts, len(parts)-1)

In [145]:
import json
with open('data.txt', 'r') as f:
    parts = json.load(f)

# combinations(parts, len(parts)-1)

pparts = []
for part in get_keys(parts):
    pparts.append(filter_parts(parts, part))

In [146]:
combs = combinations(pparts, len(pparts)-1)

In [158]:
get_attribute_sum(combs[0], 'price')

29560